## Example 7: A tissue simulation

_This example requires OpenCL-capable hardware, drivers, and an OpenCL SDK._

In this example we expand the previous simulation from a 1d fiber to a 2d slice of tissue.

As before, we'll modify the Ten Tusscher et al. model, but this time we:

- Increase epicardial Ito, and implement voltage shifts in all Ito.
- Apply heterogeneous changes to ICaL.

Instead of using a model with hardcoded changes, we'll show how to make the changes dynamically.

## Preparing the model

In [1]:
import myokit

m = myokit.load_model('models/c/tentusscher-2006.mmt')

To modify Ito, we obtain a reference to its conductance variable and then change the right-hand side (RHS) of its defining equation:

In [2]:
var = m.get('ito.gto')
print(var.rhs())

if(cell.type == 0, 0.073 [mS/uF], 0.294 [mS/uF])


We'll update this to increase epicardial (but not mid-myocardial) Ito:

In [3]:
var.set_rhs('if(cell.type == 0, 0.073, if(cell.type == 2, 0.294, 0.85))')

Next we implement voltage shifts in both of Ito's gating variables.
We start by having a look at the full Ito component:

In [4]:
print(m.get('ito').code())

[ito]
use nernst.EK as EK
use membrane.V as V
Ito = gto * r * s * (V - EK)
    in [A/F]
gto = if(cell.type == 0, 0.073, if(cell.type == 2, 0.294, 0.85))
    in [mS/uF]
dot(r) = (inf - r) / tau
    inf = 1 / (1 + exp((20 [mV] - V) / 6 [mV]))
    tau = 9.5 [ms] * exp(-(V + 40 [mV])^2 / 1800 [mV^2]) + 0.8 [ms]
        in [ms]
dot(s) = (inf - s) / tau
    inf = if(cell.type == 0, 1 / (1 + exp((V + 28 [mV]) / 5 [mV])), 1 / (1 + exp((V + 20 [mV]) / 5 [mV])))
    tau = if(cell.type == 0, 1000 [ms] * exp(-(V + 67 [mV])^2 / 1000 [mV^2]) + 8 [ms], 85 [ms] * exp(-(V + 45 [mV])^2 / 320 [mV^2]) + 5 [ms] / (1 + exp((V - 20 [mV]) / 5 [mV])) + 3 [ms])
        in [ms]




Our strategy to implement the voltage shifts will be to (1) introduce a new variable representing the shifted V and (2) to substitute V for the new variable:

In [5]:
V = m.get('membrane.V')
v20 = m.get('ito.r').add_variable('v20')
v20.set_rhs('V + 20')

# Create a mapping from "V" to "v20"
# Note that we need to use Name objects here: these represent
# variables in myokit's expression system.
sub = {myokit.Name(V): myokit.Name(v20)}

var = m.get('ito.r.inf')
var.set_rhs(var.rhs().clone(subst=sub))
var = m.get('ito.r.tau')
var.set_rhs(var.rhs().clone(subst=sub))
            
print(m.get('ito.r').code())

dot(r) = (inf - r) / tau
    inf = 1 / (1 + exp((20 [mV] - v20) / 6 [mV]))
    tau = 9.5 [ms] * exp(-(v20 + 40 [mV])^2 / 1800 [mV^2]) + 0.8 [ms]
        in [ms]
    v20 = V + 20



In [6]:
V = m.get('membrane.V')
v10 = m.get('ito.s').add_variable('v10')
v10.set_rhs('V + 10')

sub = {myokit.Name(V): myokit.Name(v10)}
var = m.get('ito.s.inf')
var.set_rhs(var.rhs().clone(subst=sub))
var = m.get('ito.s.tau')
var.set_rhs(var.rhs().clone(subst=sub))
            
print(m.get('ito.s').code())

dot(s) = (inf - s) / tau
    inf = if(cell.type == 0, 1 / (1 + exp((v10 + 28 [mV]) / 5 [mV])), 1 / (1 + exp((v10 + 20 [mV]) / 5 [mV])))
    tau = if(cell.type == 0, 1000 [ms] * exp(-(v10 + 67 [mV])^2 / 1000 [mV^2]) + 8 [ms], 85 [ms] * exp(-(v10 + 45 [mV])^2 / 320 [mV^2]) + 5 [ms] / (1 + exp((v10 - 20 [mV]) / 5 [mV])) + 3 [ms])
        in [ms]
    v10 = V + 10



Finally, we rework the equation for ICaL so that we can run with single precision.

In [7]:
var = m.get('ical.ICaL')
print(var.rhs())

ical.gCaL * ical.d * ical.f * ical.f2 * ical.fCaSS * 4 * (membrane.V - 15 [mV]) * phys.FFRT * (0.25 * calcium.CaSS * exp(2 * (membrane.V - 15 [mV]) * phys.FRT) - extra.Cao) / (exp(2 * (membrane.V - 15 [mV]) * phys.FRT) - 1)


To make it more manageable, we will split up the equation into a number of terms:

\begin{align}
I_\text{CaL} &= 
    g_\text{CaL} d f f_2 f_\text{Ca,SS} \cdot 4 (V - 15) \frac{F^2}{RT}
    \frac{
        \frac{1}{4} \text{Ca}_\text{SS} \exp(2 (V - 15) \frac{F}{RT}) - \text{Ca}_o
    }{
        \exp(2 (V - 15) \frac{F}{RT}) - 1
    } \\
    &= c_1 (V - 15) \frac{c_2 \exp(c_3 (V - 15)) - \text{Ca}_o}{\exp(c_3 (V - 15)) - 1}
\end{align}

where
\begin{align}
c_1 &= 4 g_\text{CaL} d f f_2 f_\text{Ca,SS} \frac{F^2}{RT} \\
c_2 &= \frac{1}{4} \text{Ca}_\text{SS} \\
c_3 &= 2 \frac{F}{RT} \\
\end{align}

Now we can use [l'Hopital's rule](https://en.wikipedia.org/wiki/L%27H%C3%B4pital%27s_rule) to find an equation for the limit where V approaches 15 mV.

\begin{equation}
\lim_{V \to 15} \frac{f(V)}{g(V)} = \lim_{V \to 15} \frac{f'(V)}{g'(V)}
\end{equation}

where

\begin{align}
f(V) &= c_1 (V - 15) \left[c_2 \exp(c_3 (V - 15)) - \text{Ca}_o \right] \\
f'(V) &= c_1 \left[c_2 \exp(c_3 (V - 15)) - \text{Ca}_o \right] + c_1 (V - 15) \left[c_2 c_3 \exp(c_3 (V - 15)) \right] \\
f'(15) &= c_1 \left[c_2 - \text{Ca}_o \right]
\end{align}

and

\begin{align}
g(V) &= \exp(c_3 (V - 15)) - 1 \\
g'(V) &= c_3 \exp(c_3 (V - 15)) \\
g'(15) &= c_3
\end{align}

so that $I_\text{CaL}(V = 15) = c_1 (c_2 - \text{Ca}_o) / c_3$

We can now implement this in code:

In [8]:
ical = m.get('ical.ICaL')
c1 = ical.add_variable('c1')
c1.set_rhs('gCaL * d * f * f2 * fCaSS * 4 * FFRT')
c2 = ical.add_variable('c2')
c2.set_rhs('0.25 * CaSS')
c3 = ical.add_variable('c3')
c3.set_rhs('2 * FRT')

ex1 = 'c1 * (V - 15) * (c2 * exp(c3 * (V - 15)) - Cao) / (exp(c3 * (V - 15)) - 1)'
ex2 = 'c1 * (c2 - Cao) / c3'
ical.set_rhs(f'if(abs(V - 15) > 1e-6, {ex1}, {ex2})')

print(ical.code())

ICaL = if(abs(V - 15) > 1e-6, c1 * (V - 15) * (c2 * exp(c3 * (V - 15)) - Cao) / (exp(c3 * (V - 15)) - 1), c1 * (c2 - Cao) / c3)
    in [A/F]
    c1 = gCaL * d * f * f2 * fCaSS * 4 * FFRT
    c2 = 0.25 * CaSS
    c3 = 2 * FRT



## Running the simulation

Finally, we're ready to create and run the simulation.

Note that we change ICaL conductance in this code, and use the "field" method we used before to make these changes heterogenously.

In [9]:
# Tissue dimensions
n_end = 60      # 60  0-59
n_mid = 45      # 45  60-104
n_epi = 60      # 60  105-164
ny = n_end + n_mid + n_epi
nx = 400 

# Reduced conductance
gx = 2
gy = 4

# Create a protocol
p = myokit.pacing.blocktrain(period=2000, offset=1, duration=2)
   
# Create a multi-cell simulation
s = myokit.SimulationOpenCL(m, p, ncells=(nx, ny))

# Configure the simulation
s.set_step_size(0.001)
s.set_conductance(gx, gy)

# Pacing at the endocardial side
s.set_paced_cells(nx, 2)

# Add a scalar field representing the cell type (0:endo, 1:epi, 2:mid)
import numpy as np
types = np.array([0] * n_end + [2] * n_mid + [1] * n_epi)
types = np.vstack([types] * nx).transpose()
s.set_field('cell.type', types)

# Add heterogenous ical
nx1 = int(0.1 * nx)
nx2 = nx - nx1
gcal = np.array([0.03] * nx1 + [0.012] * nx2)
gcal = np.vstack([gcal] * ny)
s.set_field('ical.gCaL', gcal)

# Run logged simulation
r = myokit.ProgressPrinter(digits=-1)
d = s.run(700, log_interval=5, log=['engine.time', 'membrane.V'], progress=r)

# Create DataBlock2d from results
b = d.block2d()

# Save to disk for inspection with Viewer
print('Saving...')
b.save('ttp-transmural-plane.zip')

[0.0 minutes] 0 % done
[1.2 minutes] 10 % done, estimated 10.7 minutes remaining
[2.4 minutes] 20 % done, estimated 9.8 minutes remaining
[3.6 minutes] 30 % done, estimated 8.4 minutes remaining
[4.7 minutes] 40 % done, estimated 7.0 minutes remaining
[5.8 minutes] 50 % done, estimated 5.8 minutes remaining
[6.9 minutes] 60 % done, estimated 4.6 minutes remaining
[8.0 minutes] 70 % done, estimated 3.4 minutes remaining
[9.1 minutes] 80 % done, estimated 2.3 minutes remaining
[10.1 minutes] 90 % done, estimated 1.1 minutes remaining
[11.2 minutes] 100 % done, estimated 0 seconds remaining
Saving...


## Viewing the result

At this point we've run a simulation and stored the results in a zip file.
But how can we view the data?

First, individual cells APs can be plotted just like we did before.
But a nicer 2d view is available by opening Myokit's "DataBlock viewer".
This can be achieved by typing `python -m myokit block` into a terminal window.


![A screenshot of the DataBlock2d viewer](block-viewer-1.png)

This can be used to view 2D colormap animations of the AP or any other logged signal.
By clicking anywhere on the animation, a time-trace at that point can be plotted, and a double-click "freezes" this trace in the lower panel.

### Creating animations

If the additional python package [moviepy](https://pypi.org/project/moviepy/) is installed, videos can also be made from the command line. E.g.
```
myokit video ttp-transmural-plane.zip membrane.V -dst video.mp4
```
creates an mp4 file [video.mp4](./video.mp4).

Other formats can be created by typing different extensions e.g. `-dst video.webm` (which works in my browser).

Showing these in notebook is tricky. This might work locally:

In [5]:
from IPython.display import Video
Video("video.webm")

This might work in github's viewer:

<video src="video.webm" width=400 />